In [1]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import json
import help_functions as hf
import numpy as np
import pandas as pd

In [2]:
# train_output = (1 - labels) * get_constr_out(output, M) + labels * get_constr_out(labels * output, M)

# TODO: add an MCM (maximum constraint module) layer here. Hierarchy constraint expressed in matrix R
# it seems the MCM layer only is used at inference? No

In [3]:
with open('training_configurations.json', 'r') as fp:
    config = json.load(fp)[str(1)]

config['batch_size'] = 1
config['nr_classes'] = 2
test = hf.get_flow(df_file=config['data_folder'] + '/test_df.json.bz2',
                   nr_classes=config['nr_classes'],
                   batch_size=config['batch_size'],
                   image_dimension=config['image_dimension'])

Found 50000 non-validated image filenames belonging to 40 classes.


/home/matvieir/wiki_image_classification/src/classification/help_functions.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x_labels['labels'] = df['labels'].apply(lambda labels_list: [label for label in labels_list if label in top_classes])


Found 31925 validated image filenames belonging to 2 classes.


In [10]:
# Read dataframe 
df = pd.read_json(config['data_folder'] + '/test_df.json.bz2')
df.head(2)

,id,title,url,categories,labels,naive_labels
1596792,3915827,Paysage avec deux hommes assis près d'un taill...,a/a8/Paysage_avec_deux_hommes_assis_prÃ¨s_d'un...,"[Italian drawings in the Louvre, Everhard Jabach]","[People, Culture, Society]","[Culture, Places]"
992788,22139193,Bregoc.JPG,5/55/Bregoc.JPG,[Zelengora],[Places],[Places]


In [11]:
# Transform the labels to tf.ragged.constant to not get "ValueError: Can't convert non-rectangular Python sequence to Tensor" 
# (https://stackoverflow.com/questions/56304986/valueerror-cant-convert-non-rectangular-python-sequence-to-tensor)
test_ds = tf.data.Dataset.from_tensor_slices((df.url.values, tf.ragged.constant(df.labels.values))).batch(config['batch_size'])

In [12]:
class CustomModel(keras.Model):
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True) # forward pass
            # Compute loss
            bce = tf.keras.losses.BinaryCrossentropy(from_logits=False)
            loss = bce(y, y_pred)
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

def get_uncompiled_model():
    basemodel = EfficientNetB2(include_top=False, weights=None, classes=20, input_shape=(32, 32, 3))
    inputs = Input(shape=(32, 32, 3))
    x = basemodel(inputs)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(20, activation='sigmoid')(x)
    model = CustomModel(inputs=inputs, outputs=outputs)
    return model

def get_compiled_model():
    model = get_uncompiled_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    return model

# model = get_compiled_model()
# model.fit(test, verbose=1, epochs=1)

In [4]:
basemodel = EfficientNetB2(include_top=False, weights=None, classes=config['nr_classes'], input_shape=(config['image_dimension'], config['image_dimension'], 3))
inputs = Input(shape=(config['image_dimension'], config['image_dimension'], 3))
x = basemodel(inputs)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(config['nr_classes'], activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)
# model.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['categorical_accuracy'])
loss_fn = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()
metrics = tf.metrics.CategoricalAccuracy()

In [9]:
# M_ij = 1 if the class i is a subclass of the class j. I.e.: row i, column j is 1.
# - note that the diagonal must always be 0, as no class is a subclass of itseld
# - note also that if M_ij is 1 then M_ji is necessarily 0 as the subclass relation is not reflexive
mask = np.array([[0, 0],
                 [1, 0]], dtype=np.float32) # i.e. the second and the third classes are children of the first

def max_constrain(output, mask):
    "Constrains the output given the hierarchy expressed by the mask."
    batch_size = len(output) # or output.shape[0]
    nr_classes = mask.shape[0] # = mask.shape[1] = output.shape[1]


    output = tf.expand_dims(output, axis=1)
    print(output)

    batch_output = tf.broadcast_to(output, [batch_size, nr_classes, nr_classes]) # this is H in the MCM equation. Looks good, i.e., as a replication of the predictions in a (nr_classes X nr_classes) matrix
    batch_mask = tf.broadcast_to(mask, [batch_size, nr_classes, nr_classes])
    temp = batch_output * batch_mask
    print(f'batch_output:\n {batch_output}')
    print(f'temp:\n {temp}')

    constrained_output = tf.math.reduce_max(temp, axis=2)
    print(constrained_output)
    return
    return constrained_output

# @tf.function
def train_step(inputs, labels):
    with tf.GradientTape() as tape:
        y_probs = model(inputs, training=True)

        # Extra steps for coherent HMC:
        # 1. max constraint module        
        term_1 = (1 - labels) * max_constrain(y_probs, mask)
        term_2 = labels * max_constrain(labels * y_probs, mask)
        y_probs_constrained = term_1 + term_2
        # TODO: control if the outputs are indeed coherent, i.e. that a parent class always has greater or equal probability as its children

        # 2. modify what is sent to binary cross-entropy function (not anymore y_true and y_probs)
        # loss_value = model.compiled_loss(labels, y_probs_constrained)
        loss_value = loss_fn(labels, y_probs_constrained)

    grads = tape.gradient(loss_value, model.trainable_weights)
    # model.optimizer.apply_gradients(zip(grads, model.trainable_weights))
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    metrics.update_state(labels, y_probs_constrained)
    return loss_value
    
epochs = 1
for epoch in range(epochs):
    print(f'Start of epoch {epoch}')

    # Iterate over the batches of the dataset
    for step, (x_batch_train, y_batch_train) in enumerate(test):
        loss_value = train_step(x_batch_train, y_batch_train)

        # Log every 200 batches.
        if step % 200 == 0:
            print(f"Training loss (for one batch) at step {step}: {float(loss_value):.4f}")
            print(f"Seen so far: {(step + 1) * config['batch_size']} samples")

    # Display metrics at the end of each epoch.
    train_acc = metrics.result()
    # print(f"Training acc over epoch: {float(train_acc)}")

    # Reset training metrics at the end of each epoch
    metrics.reset_states()

Start of epoch 0
tf.Tensor([[[0.81161976 0.70110446]]], shape=(1, 1, 2), dtype=float32)
batch_output:
 [[[0.81161976 0.70110446]
  [0.81161976 0.70110446]]]
temp:
 [[[0.         0.        ]
  [0.81161976 0.        ]]]
tf.Tensor([[0.         0.81161976]], shape=(1, 2), dtype=float32)


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'